# 12.1 子类化内置类型很麻烦

内置类型（使用C语言编写）不会调用用户定义的类覆盖的特殊方法

In [1]:
# 内置类型 dict 的 __init__ 和 __update__ 方法会忽略我们覆盖的 __setitem__ 方法
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

# 继承自 dict 的 __init__ 方法线然忽略了我们覆盖的 __setitem__ 方法： 'one' 的值没有出现重复
dd = DoppelDict(one = 1)
dd


{'one': 1}

In [2]:
# [] 运算会调用我们覆盖的 __setitem__ 方法
dd['two'] = 2

In [3]:
# 继承自 dict 的 update 方法也不会使用我们覆盖的 __setitem__ 方法
dd.update(three = 3)

直接子类化内部类型容易出错，用户自己定义的类应该继承 collections 模块，例如 UserDict、UserList 和 UserString ，这些类做了特殊设计，因此易于扩展。  
这些问题只会发生在 C 语言实现的内置类型内部的方法委托上，而且之影响直接继承内置类型的用户自定义类，如果子类化使用的是 Python 编写的类，就不会受此影响。

# 12.2 多重继承和方法解析顺序

<img src=.\Figure12-1.png >

In [2]:
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

注意：B 和 C 都实现了 pong 方法，二者之间唯一的区别是，C.pong 方法输出的是大写的 PONG

在 D 的实例上调用 D.pong() 方法的话，运行的是那个 pong 方法呢

In [6]:
# 对类D的实例的pong方法调用的两种形式
d = D()

# 直接调用 d.pong() 运行的是 B 类中的版本
d.pong()

# 超类中的方法都可以直接调用，此视要把实例作为显示参数传入
C.pong(d)

pong: <__main__.D object at 0x000001E98E1479D0>
PONG: <__main__.D object at 0x000001E98E1479D0>


Python 能区分 d.pong() 调用的是哪个方法，是因为 Python 会按照特定的顺序遍历继承图。这个顺序叫方法解析顺序（Method Resolution Order，MRO）。类都有一个名为 \_\_mro__ 的属性，它的值是一个元组，按照方法解析顺序列出各个超类

In [7]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

若想把方法调用委托给超类，推荐的方式是使用内置的 super() 函数，在 Python3 中，这种方式变得更容易了，如示例 12-4 中 D 类的 pingpong 方法表示  
然而，有时可能需要绕过方法解析顺序，直接调用某个超类的方法 —— 这样做有时更方便。所以 D.ping 方法可以这样写

In [ ]:
def ping(self):
    A.ping(self)
    print('post-ping', self)

注意，直接在类上调用实例方法时，必须显式的传入 self 参数，因为这样访问的是 未绑定的方法  
然而，使用 super() 最安全，也不易过时。调用框架或不受自己控制的类层次结构中的方法时，尤其适合使用 super()。

In [3]:
# 使用 super() 函数调用 ping 方法
d = D()
d.ping()

ping: <__main__.D object at 0x000002295D9B6430>
post-ping: <__main__.D object at 0x000002295D9B6430>


D 类的 ping 方法做了两次调用  
第一个调用是 super().ping(); super 函数把 ping 调用委托给 A 类；这一行由 A.ping 输出
第二个调用是 print('post-ping', self)

In [4]:
# pingpong 的 5 个调用
d.pingpong()

ping: <__main__.D object at 0x000002295D9B6430>
post-ping: <__main__.D object at 0x000002295D9B6430>
ping: <__main__.D object at 0x000002295D9B6430>
pong: <__main__.D object at 0x000002295D9B6430>
pong: <__main__.D object at 0x000002295D9B6430>
PONG: <__main__.D object at 0x000002295D9B6430>


第一个是 self.ping() 调用 D 类的 ping 方法  
第二个是 super().ping() 跳过 D 类的 ping 方法，找到 D 类的 ping 方法  
第三个是 self.pong() 根据 \_\_mro__ 找到 B 类实现的 ping 方法  
第四个是 super().pong() 根据 \_\_mro__ 找到 B 类实现的 ping 方法  
第五个是 C.pong(self) 显示的调用超类 C 的 pong 方法  

方法解析顺序不仅考虑继承图，还考虑子类声明中列出超类的顺序，所以在 D 的超类顺序中， B 在 C 之前  
分析类时，可以查看 \_\_mro__ 以便了解具体情况

In [5]:
bool.__mro__

(bool, int, object)

In [6]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [8]:
import numbers
print_mro(numbers.Integral)

class test:
    pass
print_mro(test)

Integral, Rational, Real, Complex, Number, object
test, object


# 12.3 多重继承的真实应用

麻了，图太多，看书吧